# Medi-Cal Enrollment AI Analytics

In this analysis of Medi-Cal enrollment data, we leveraged a suite of Snowflake AI SQL features to rapidly transform raw monthly enrollment figures into clear, actionable insights. We began by aggregating enrollment counts by eligibility group over a six-month window and computing the percentage change for each group. To classify each trend as “Increasing,” “Stable,” or “Decreasing,” we used the AI_CLASSIFY function, which applies a lightweight model to label patterns in the data for easy interpretation. Next, we enriched our output with natural-language summaries generated by AI_COMPLETE (using the Claude-3-7-sonnet model) driven by carefully crafted PROMPT templates, providing plain-English explanations of each group’s trajectory. Although not shown here, similar patterns could be extended with AI_AGG and AI_SUMMARIZE for more advanced aggregation or narrative generation.

For visualization, we turned our classified and summarized results into a horizontal diverging bar chart using Altair: positive percentage changes are shaded in blue, negative in gray, and the bars are sorted to emphasize the most dramatic shifts. The chart’s dynamic title reflects the analysis date range, and its clean, professional styling makes it easy for stakeholders to spot which eligibility groups are growing or shrinking at a glance. By combining AI-driven classification, summarization, and traditional SQL aggregations with intuitive visual encoding, this pipeline delivers a comprehensive, end-to-end view of enrollment trends.

In [ ]:
# Import python packages
import pandas as pd
import altair as alt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# AI_CLASSIFY – Trend Tagging and AI_COMPLETE – Summary Generation

This notebook cell performs two AI-powered tasks on Medi-Cal enrollment data:

1. **Data Aggregation**  
   - Collects each eligibility group’s last six months of `NUMBER_OF_ENROLLEES` into a comma-delimited string (`ENROLLEE_COUNTS`).  
   - Records the first and last reporting dates (`RP_FIRST`, `RP_LAST`).

2. **Trend Classification**  
   - Uses **AI_CLASSIFY** with a fixed label set:
     - **Increasing** – enrollee counts are rising  
     - **Stable**     – enrollee counts show little or no change  
     - **Decreasing** – enrollee counts are falling  
   - Applies a **PROMPT** template to the `ENROLLEE_COUNTS` string and returns the most appropriate trend label for each group.

3. **Natural-Language Summary**  
   - Uses **AI_COMPLETE** (model `claude-3-7-sonnet`) with a **PROMPT** to generate a single plain-English summary per group.  
   - The summary includes the first and last values and the calculated percentage change over the period.

The result is a table `ENROLLMENT_TRENDS` containing:
- The original `ELIGIBILITY_GROUP`  
- `TREND_CLASSIFICATION` from **AI_CLASSIFY**  
- `SUMMARY` from **AI_COMPLETE**  
- `RP_FIRST`, `RP_LAST`, and the raw `ENROLLEE_COUNTS` for reference  


In [ ]:
CREATE OR REPLACE TABLE ENROLLMENT_TRENDS AS
(
  WITH ENROLLMENT_DATA AS (
    SELECT
      ELIGIBILITY_GROUP,
      DATEADD(month, -5, MAX(REPORTING_PERIOD)) AS RP_FIRST,
      MAX(REPORTING_PERIOD)                           AS RP_LAST,
      LISTAGG(NUMBER_OF_ENROLLEES, ',') 
        WITHIN GROUP (ORDER BY REPORTING_PERIOD)      AS ENROLLEE_COUNTS,
      # Build the raw prompt string for classification
      PROMPT(
        'The enrollee counts for {0} over the past six months between {1} and {2} are: {3}. '
        || 'Label the trend as Increasing, Stable, or Decreasing.',
        ELIGIBILITY_GROUP,
        DATEADD(month, -5, MAX(REPORTING_PERIOD)),
        MAX(REPORTING_PERIOD),
        LISTAGG(NUMBER_OF_ENROLLEES, ',') WITHIN GROUP (ORDER BY REPORTING_PERIOD)
      )                                              AS TREND_PROMPT
    FROM MEDI_CAL_ELIGIBILITY.ELIGIBILITY.MEDI_CAL_ENROLLMENT  
    WHERE REPORTING_PERIOD BETWEEN
      DATEADD(month, -5, (SELECT MAX(REPORTING_PERIOD) FROM MEDI_CAL_ELIGIBILITY.ELIGIBILITY.MEDI_CAL_ENROLLMENT))
      AND (SELECT MAX(REPORTING_PERIOD) FROM MEDI_CAL_ELIGIBILITY.ELIGIBILITY.MEDI_CAL_ENROLLMENT)
    GROUP BY ELIGIBILITY_GROUP
  )

  SELECT
    ELIGIBILITY_GROUP,

    # Generate a plain-English summary of first/last counts and percent change
    AI_COMPLETE(
      'claude-3-7-sonnet', 
      PROMPT(
        'Summarize the trend for {0} over the past six months: first count = {1}, '
        || 'last count = {2}, and percent change in a single brief statement.',
        ELIGIBILITY_GROUP,
        RP_FIRST,
        RP_LAST
      )
    )                                             AS SUMMARY,

    # Classify the trend label using the TREND_PROMPT
    AI_CLASSIFY(
      TREND_PROMPT,
      [
        { 'label': 'Increasing', 'description': 'The enrollee counts are increasing.' },
        { 'label': 'Stable',     'description': 'The enrollee counts are stable.' },
        { 'label': 'Decreasing', 'description': 'The enrollee counts are decreasing.' }
      ],
      {
        'task_description': 'Determine whether the given enrollee counts string indicates an increasing, stable, or decreasing trend.',
        'output_mode':      'single'
      }
    )                                             AS TREND_CLASSIFICATION,

    RP_FIRST,
    RP_LAST,
    ENROLLEE_COUNTS

  FROM ENROLLMENT_DATA
);


In [ ]:
# Show trends   
SELECT ELIGIBILITY_GROUP, SUMMARY, TREND_CLASSIFICATION:labels[0]::STRING AS TREND  FROM ENROLLMENT_TRENDS;

## Enrollment Trend Visualization

This cell transforms the precomputed `ENROLLMENT_TRENDS` table into an interactive, horizontal diverging bar chart that highlights percentage changes in Medi-Cal enrollment by eligibility group over a six-month period.

1. **Data Retrieval**  
   - Reads `ELIGIBILITY_GROUP`, `ENROLLEE_COUNTS`, `RP_FIRST`, and `RP_LAST` from the `ENROLLMENT_TRENDS` table in Snowflake.

2. **Parsing & Cleanup**  
   - Splits the comma-delimited `ENROLLEE_COUNTS` into the first and last monthly values.  
   - Converts these values to integers and drops any malformed rows.

3. **Calculations**  
   - Computes the percentage change between the first and last counts.  
   - Formats the change as a string (e.g. `+12.34%`) for easy reading.

4. **Date Range Extraction**  
   - Parses `RP_FIRST` and `RP_LAST` into Python `date` objects.  
   - Determines the overall reporting window for use in chart titles.

5. **Sorting**  
   - Orders the DataFrame by the numeric percentage change to prepare for the diverging layout.

6. **Visualization**  
   - Uses Altair to render a horizontal bar chart where:
     - Bars to the right (positive change) are colored blue.  
     - Bars to the left (negative change) are colored gray.  
   - The x-axis title dynamically displays the date range.  
   - The y-axis lists eligibility groups in order of change magnitude.

7. **Outcome**  
   - Provides a clear, at-a-glance view of which eligibility groups experienced the largest increases or decreases in enrollment, aiding data-driven decision making.  


In [ ]:
# 1. Fetch data including RP_FIRST and RP_LAST
df = session.sql("""
  SELECT ELIGIBILITY_GROUP, ENROLLEE_COUNTS, RP_FIRST, RP_LAST
  FROM ENROLLMENT_TRENDS
""").to_pandas()

# 2. Vectorized parse of ENROLLEE_COUNTS into first and last counts
counts_df = df['ENROLLEE_COUNTS'].str.split(',', expand=True)
df['first_count'] = counts_df.iloc[:, 0].astype(int, errors='raise')
df['last_count']  = counts_df.iloc[:, -1].astype(int, errors='raise')

# 3. Compute percentage change (numeric and formatted)
df['pct_change_num'] = (df['last_count'] - df['first_count']) / df['first_count'] * 100
df['pct_change']     = df['pct_change_num'].map("{:+.2f}%".format)

# 4. Parse dates and determine the overall range
df['RP_FIRST'] = pd.to_datetime(df['RP_FIRST']).dt.date
df['RP_LAST']  = pd.to_datetime(df['RP_LAST']).dt.date
start_date = df['RP_FIRST'].min().isoformat()
end_date   = df['RP_LAST'].max().isoformat()

# 5. Sort for diverging order
df = df.sort_values('pct_change_num')

# 6. Display DataFrame for verification (Snowflake notebook will render it automatically)
df

# 7. Create horizontal diverging bar chart with dynamic x-axis title
x_title = f'Percentage Change ({start_date} to {end_date})'

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('pct_change_num:Q', title=x_title),
    y=alt.Y('ELIGIBILITY_GROUP:N', sort=list(df['ELIGIBILITY_GROUP']), title='Eligibility Group'),
    color=alt.condition(
        alt.datum.pct_change_num > 0,
        alt.value('steelblue'),
        alt.value('lightgray')
    )
).properties(
    width=600,
    height=300,
    title='Enrollment Percentage Change'
)

chart


## Enrollment Trend & Share Analysis

This notebook cell transforms the precomputed `ENROLLMENT_TRENDS` table into two interactive pie charts that reveal:

1. **Share of Total Enrollees by Group**  
   - **Data Loading:** Retrieves `ELIGIBILITY_GROUP`, the comma-delimited `ENROLLEE_COUNTS`, and `RP_LAST` (last reporting date).  
   - **Vectorized Parsing:** Splits the counts string into first/last monthly values for each group.  
   - **Percentage Change:** Computes numeric and formatted percent change between first and last counts.  
   - **Dynamic Date Label:** Derives the month-year of the final period (e.g. “Dec 2024”) for the chart title.  
   - **Pie Chart:** Displays each group’s share of the total enrollment in the final month, with tooltips for raw counts and percentages.

2. **Proportion of Groups by Trend Classification**  
   - **Trend Assignment:** Classifies each group as **Increasing**, **Stable**, or **Decreasing** based on sign of the percent change (0% → “Stable”).  
   - **Aggregation:** Counts how many groups fall into each trend category and computes their share of the total number of groups.  
   - **Pie Chart:** Shows the relative proportion of “Increasing,” “Stable,” and “Decreasing” groups, with tooltips for group counts and percentages.

### Key Efficiency Improvements
- **Vectorized operations** replace per-row `apply` calls for parsing and classification.  
- **Dynamic titles** ensure the charts automatically reflect the actual data range.  
- **Single-pass aggregations** and in-memory filtering minimize data movement between Snowflake and Python.  

By combining concise data transformation with clean, side-by-side visualizations, this cell provides an at-a-glance view of both enrollment distribution and trend breakdown across eligibility groups.  


In [ ]:
# 1. Load group + counts
df = session.sql("""
  SELECT ELIGIBILITY_GROUP, ENROLLEE_COUNTS, RP_LAST
  FROM ENROLLMENT_TRENDS
""").to_pandas()

# 2. Vectorized parsing of counts
counts_df = df['ENROLLEE_COUNTS'].str.split(',', expand=True)
df['first_count'] = counts_df.iloc[:,0].astype(int, errors='coerce')
df['last_count']  = counts_df.iloc[:,-1].astype(int, errors='coerce')
df = df.dropna(subset=['first_count','last_count'])

# 3. % change & classification (with Stable)
df['pct_change_num'] = (df['last_count'] - df['first_count']) / df['first_count'] * 100
df['pct_change']     = df['pct_change_num'].map("{:+.2f}%".format)
df['trend'] = df['pct_change_num'].apply(
    lambda x: 'Increasing' if x>0 else ('Stable' if x==0 else 'Decreasing')
)

# 4. Pie 1: Share of last_count by group (dynamic title)
dec_label = pd.to_datetime(df['RP_LAST']).dt.strftime('%b %Y').max()
df['share'] = df['last_count'] / df['last_count'].sum()
pie1 = alt.Chart(df).mark_arc().encode( ... ).properties(
    title=f'Share of Total Enrollees by Eligibility Group ({dec_label})'
)

# 5. Pie 2: Proportion of groups by trend
df_trend = df['trend'].value_counts(normalize=True).reset_index()
df_trend.columns = ['trend','share']
df_trend['count'] = df['trend'].value_counts().values
pie2 = alt.Chart(df_trend).mark_arc().encode( ... )

# 6. Render side-by-side
pie1 | pie2